In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_labels = path.join(root_folder,'data/train_labels.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 25;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [5]:
mean_std = torch.tensor(np.load(path_to_mean_std))
mean_train = mean_std[0].repeat(train_data.__len__());
std_train = mean_std[1].repeat(train_data.__len__());
mean_test = mean_std[0].repeat(test_data.__len__());
std_test = mean_std[1].repeat(test_data.__len__());
transform = transforms.Compose([transforms.Normalize(mean_train, std_train)])
train_data.data = transform(train_data.data)
transform = transforms.Compose([transforms.Normalize(mean_test, std_test)])
test_data.data = transform(test_data.data)

/home/vtpc/.local/lib/python3.5/site-packages/torchvision/transforms/functional.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean = torch.tensor(mean, dtype=torch.float32)
/home/vtpc/.local/lib/python3.5/site-packages/torchvision/transforms/functional.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  std = torch.tensor(std, dtype=torch.float32)


In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)




In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/11127614 (0%)]	Loss: 0.71531581878662109375
Train Epoch: 0 [450000/11127614 (4%)]	Loss: 0.24527545273303985596
Train Epoch: 0 [900000/11127614 (8%)]	Loss: 0.15859551727771759033
Train Epoch: 0 [1350000/11127614 (12%)]	Loss: 0.11640586704015731812
Train Epoch: 0 [1800000/11127614 (16%)]	Loss: 0.08599976450204849243
Train Epoch: 0 [2250000/11127614 (20%)]	Loss: 0.06449190527200698853
Train Epoch: 0 [2700000/11127614 (24%)]	Loss: 0.05436292663216590881
Train Epoch: 0 [3150000/11127614 (28%)]	Loss: 0.04954466223716735840
Train Epoch: 0 [3600000/11127614 (32%)]	Loss: 0.04053021594882011414
Train Epoch: 0 [4050000/11127614 (36%)]	Loss: 0.03947626054286956787
Train Epoch: 0 [4500000/11127614 (40%)]	Loss: 0.03886003792285919189
Train Epoch: 0 [4950000/11127614 (44%)]	Loss: 0.03125013038516044617
Train Epoch: 0 [5400000/11127614 (49%)]	Loss: 0.03179214149713516235
Train Epoch: 0 [5850000/11127614 (53%)]	Loss: 0.03168831020593643188
Train Epoch: 0 [6300000/11127614 (57%)]	Loss:

Train Epoch: 4 [6300000/11127614 (57%)]	Loss: 0.02108371444046497345
Train Epoch: 4 [6750000/11127614 (61%)]	Loss: 0.02148062549531459808
Train Epoch: 4 [7200000/11127614 (65%)]	Loss: 0.02102841064333915710
Train Epoch: 4 [7650000/11127614 (69%)]	Loss: 0.02101631090044975281
Train Epoch: 4 [8100000/11127614 (73%)]	Loss: 0.02123872563242912292
Train Epoch: 4 [8550000/11127614 (77%)]	Loss: 0.02080720663070678711
Train Epoch: 4 [9000000/11127614 (81%)]	Loss: 0.02047968842089176178
Train Epoch: 4 [9450000/11127614 (85%)]	Loss: 0.01951016485691070557
Train Epoch: 4 [9900000/11127614 (89%)]	Loss: 0.01834851317107677460
Train Epoch: 4 [10350000/11127614 (93%)]	Loss: 0.02146232500672340393
Train Epoch: 4 [10800000/11127614 (97%)]	Loss: 0.01777231693267822266

Test set: Average loss: 0.00000120547724691278, Accuracy: 2764169/2781904 (99.36248698732954665047%)

Train Epoch: 5 [0/11127614 (0%)]	Loss: 0.02020689658820629120
Train Epoch: 5 [450000/11127614 (4%)]	Loss: 0.01718586497008800507
Train E

Train Epoch: 9 [450000/11127614 (4%)]	Loss: 0.01893894188106060028
Train Epoch: 9 [900000/11127614 (8%)]	Loss: 0.02013792470097541809
Train Epoch: 9 [1350000/11127614 (12%)]	Loss: 0.01892803423106670380
Train Epoch: 9 [1800000/11127614 (16%)]	Loss: 0.01970461010932922363
Train Epoch: 9 [2250000/11127614 (20%)]	Loss: 0.01981121860444545746
Train Epoch: 9 [2700000/11127614 (24%)]	Loss: 0.02270905114710330963
Train Epoch: 9 [3150000/11127614 (28%)]	Loss: 0.01929604448378086090
Train Epoch: 9 [3600000/11127614 (32%)]	Loss: 0.02013908885419368744
Train Epoch: 9 [4050000/11127614 (36%)]	Loss: 0.02070284821093082428
Train Epoch: 9 [4500000/11127614 (40%)]	Loss: 0.02216320857405662537
Train Epoch: 9 [4950000/11127614 (44%)]	Loss: 0.01828582584857940674
Train Epoch: 9 [5400000/11127614 (49%)]	Loss: 0.01555854547768831253
Train Epoch: 9 [5850000/11127614 (53%)]	Loss: 0.01946459710597991943
Train Epoch: 9 [6300000/11127614 (57%)]	Loss: 0.01602011173963546753
Train Epoch: 9 [6750000/11127614 (61%)

Train Epoch: 13 [6300000/11127614 (57%)]	Loss: 0.01928168907761573792
Train Epoch: 13 [6750000/11127614 (61%)]	Loss: 0.02141595445573329926
Train Epoch: 13 [7200000/11127614 (65%)]	Loss: 0.01926963590085506439
Train Epoch: 13 [7650000/11127614 (69%)]	Loss: 0.01934945955872535706
Train Epoch: 13 [8100000/11127614 (73%)]	Loss: 0.02093685045838356018
Train Epoch: 13 [8550000/11127614 (77%)]	Loss: 0.01596526987850666046
Train Epoch: 13 [9000000/11127614 (81%)]	Loss: 0.01896082423627376556
Train Epoch: 13 [9450000/11127614 (85%)]	Loss: 0.02173453941941261292
Train Epoch: 13 [9900000/11127614 (89%)]	Loss: 0.02144385129213333130
Train Epoch: 13 [10350000/11127614 (93%)]	Loss: 0.02010439522564411163
Train Epoch: 13 [10800000/11127614 (97%)]	Loss: 0.01997333578765392303

Test set: Average loss: 0.00000118879449928500, Accuracy: 2764376/2781904 (99.36992793424934689028%)

Train Epoch: 14 [0/11127614 (0%)]	Loss: 0.01887667924165725708
Train Epoch: 14 [450000/11127614 (4%)]	Loss: 0.019585493952035

Train Epoch: 18 [0/11127614 (0%)]	Loss: 0.02030463330447673798
Train Epoch: 18 [450000/11127614 (4%)]	Loss: 0.01774642057716846466
Train Epoch: 18 [900000/11127614 (8%)]	Loss: 0.01801571808755397797
Train Epoch: 18 [1350000/11127614 (12%)]	Loss: 0.02391868457198143005
Train Epoch: 18 [1800000/11127614 (16%)]	Loss: 0.01806614734232425690
Train Epoch: 18 [2250000/11127614 (20%)]	Loss: 0.01916933804750442505
Train Epoch: 18 [2700000/11127614 (24%)]	Loss: 0.01899557933211326599
Train Epoch: 18 [3150000/11127614 (28%)]	Loss: 0.02001644298434257507
Train Epoch: 18 [3600000/11127614 (32%)]	Loss: 0.01561331562697887421
Train Epoch: 18 [4050000/11127614 (36%)]	Loss: 0.01955620571970939636
Train Epoch: 18 [4500000/11127614 (40%)]	Loss: 0.01965249143540859222
Train Epoch: 18 [4950000/11127614 (44%)]	Loss: 0.01823276281356811523
Train Epoch: 18 [5400000/11127614 (49%)]	Loss: 0.01832561194896697998
Train Epoch: 18 [5850000/11127614 (53%)]	Loss: 0.02007451839745044708
Train Epoch: 18 [6300000/111276

Train Epoch: 22 [5400000/11127614 (49%)]	Loss: 0.02160924859344959259
Train Epoch: 22 [5850000/11127614 (53%)]	Loss: 0.01888252422213554382
Train Epoch: 22 [6300000/11127614 (57%)]	Loss: 0.02308941073715686798
Train Epoch: 22 [6750000/11127614 (61%)]	Loss: 0.01911592110991477966
Train Epoch: 22 [7200000/11127614 (65%)]	Loss: 0.02119913510978221893
Train Epoch: 22 [7650000/11127614 (69%)]	Loss: 0.02088253386318683624
Train Epoch: 22 [8100000/11127614 (73%)]	Loss: 0.01716028153896331787
Train Epoch: 22 [8550000/11127614 (77%)]	Loss: 0.02077444456517696381
Train Epoch: 22 [9000000/11127614 (81%)]	Loss: 0.01810018718242645264
Train Epoch: 22 [9450000/11127614 (85%)]	Loss: 0.01806742697954177856
Train Epoch: 22 [9900000/11127614 (89%)]	Loss: 0.01727173663675785065
Train Epoch: 22 [10350000/11127614 (93%)]	Loss: 0.01888789795339107513
Train Epoch: 22 [10800000/11127614 (97%)]	Loss: 0.01866983063519001007

Test set: Average loss: 0.00000118752654998389, Accuracy: 2764331/2781904 (99.368310337

In [8]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00000118772265977896, Accuracy: 2764353/2781904 (99.36910116236937540179%)

